# 🔗 Splink Demo: Probabilistic Record Linkage in Google Colab

This notebook demonstrates how to use [Splink](https://github.com/moj-analytical-services/splink) for probabilistic record linkage using a synthetic dataset. We'll use the DuckDB backend for simplicity.

In [ ]:
!pip install "splink[duckdb]" -q

In [ ]:
import pandas as pd
import numpy as np
from splink.duckdb.linker import DuckDBLinker
import matplotlib.pyplot as plt
import seaborn as sns


## 🧪 Create Synthetic Dataset

In [ ]:
np.random.seed(42)

# Base dataset
first_names = ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve']
last_names = ['Smith', 'Johnson', 'Williams', 'Brown', 'Jones']
domains = ['example.com', 'test.org', 'demo.net']

def generate_record(i):
    fn = np.random.choice(first_names)
    ln = np.random.choice(last_names)
    email = f"{fn.lower()}.{ln.lower()}@{np.random.choice(domains)}"
    return {"unique_id": i, "first_name": fn, "last_name": ln, "email": email}

records = [generate_record(i) for i in range(100)]
df = pd.DataFrame(records)

# Introduce duplicates with slight variations
duplicates = df.sample(10).copy()
duplicates["first_name"] = duplicates["first_name"].apply(lambda x: x if np.random.rand() > 0.5 else x[:-1])
duplicates["last_name"] = duplicates["last_name"].apply(lambda x: x if np.random.rand() > 0.5 else x[:-1])
duplicates["email"] = duplicates["email"].apply(lambda x: x.replace('@', '_dup@'))

df_combined = pd.concat([df, duplicates], ignore_index=True)
df_combined.reset_index(drop=True, inplace=True)
df_combined.head()


## ⚙️ Configure Splink

In [ ]:
settings = {
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        "l.first_name = r.first_name",
        "l.last_name = r.last_name"
    ],
    "comparisons": [
        {
            "col_name": "first_name",
            "comparison_levels": [
                {"sql_condition": "l.first_name = r.first_name", "label": "Exact match"},
                {"sql_condition": "levenshtein(l.first_name, r.first_name) <= 1", "label": "Fuzzy match"},
                {"sql_condition": "else", "label": "No match"}
            ]
        },
        {
            "col_name": "last_name",
            "comparison_levels": [
                {"sql_condition": "l.last_name = r.last_name", "label": "Exact match"},
                {"sql_condition": "else", "label": "No match"}
            ]
        },
        {
            "col_name": "email",
            "comparison_levels": [
                {"sql_condition": "l.email = r.email", "label": "Exact match"},
                {"sql_condition": "else", "label": "No match"}
            ]
        }
    ]
}


## 🔗 Initialize Linker and Train Model

In [ ]:
linker = DuckDBLinker(df_combined, settings)
linker.estimate_u_using_random_sampling(max_pairs=1e6)
linker.estimate_probability_two_random_records_match()
linker.estimate_parameters_using_expectation_maximisation()


## 📊 Predict Matches and View Results

In [ ]:
df_predictions = linker.predict()
df_predictions = df_predictions.as_pandas_dataframe()
df_predictions[['unique_id_l', 'unique_id_r', 'match_probability']].sort_values(by='match_probability', ascending=False).head(10)


## 📈 Visualize Match Probabilities

In [ ]:
sns.histplot(df_predictions['match_probability'], bins=20, kde=True)
plt.title("Distribution of Match Probabilities")
plt.xlabel("Match Probability")
plt.ylabel("Frequency")
plt.show()
